Auto-Clean-Test

In [1]:
import logging
import os
import shutil
from streamline.runners.auto_runner import AutoRunner

/Users/gabrielketron/AutoTest/AutoTestEnv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
autotest = AutoRunner(dataset_names=['hcc-data_example'], clean=False)

type: 'AutoRunner' object has no attribute 'clean'

In [ ]:
autotest.run()